In [ ]:
import os
import sys

# We can commnent this three lines of code because we aer now using wheels and all is setup to work with wheels and we no longuer need these lines of code.
#current_dir = os.getcwd()
#project_root = os.path.abspath(os.path.join(current_dir, "..", "..", ".."))

#sys.path.append(project_root)

In [ ]:
#from src.citibike.citibike_utils import get_trip_duration_mins
#from src.utils.datetime_utils import timestamp_to_date_col
#from pyspark.sql.functions import create_map, lit

# This is used when we use python wheels...

from citibike.citibike_utils import get_trip_duration_mins
from utils.datetime_utils import timestamp_to_date_col
from pyspark.sql.functions import create_map, lit

In [ ]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

In [ ]:
df = spark.read.table(f"{catalog}.01_bronze.jc_citibike")

In [6]:
df = get_trip_duration_mins(spark, df, "started_at", "ended_at", "trip_duration_mins")

In [8]:
df = timestamp_to_date_col(spark, df, "started_at", "trip_start_date")

In [ ]:
df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

In [11]:
df = df.select(
    "ride_id",
    "trip_start_date",
    "started_at",
    "ended_at",
    "start_station_name",
    "end_station_name",
    "trip_duration_mins",
    "metadata"
    )

In [ ]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.02_silver.jc_citibike")